In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Install if not already
!pip install ydata-profiling --quiet

import pandas as pd
from ydata_profiling import ProfileReport

# Load dataset
train = pd.read_csv("/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/train.csv")

# # Generate report
# profile = ProfileReport(train, title="Air Quality EDA Report", explorative=True)

# # Display in notebook
# # Display in notebook
# profile.to_notebook_iframe()

/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/sample_submission.csv
/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/train.csv
/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/test.csv


In [28]:
!pip install qolmat --quiet
import pandas as pd
from qolmat.imputations import imputers

train = pd.read_csv("/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/train.csv")
test = pd.read_csv("/kaggle/input/x-hec-ts-2025-26-predicting-air-quality-in-paris/test.csv")


train['id'] = pd.to_datetime(train['id'])  # adjust column name
test['id'] = pd.to_datetime(test['id'])
train = train.sort_values(['id'])
test = test.sort_values(['id'])
train = train.set_index('id')
test = test.set_index('id')

# Check structure
train.info()
train.head()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40991 entries, 2020-01-01 00:00:00 to 2024-09-03 22:00:00
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   valeur_NO2   37694 non-null  float64
 1   valeur_CO    28462 non-null  float64
 2   valeur_O3    40298 non-null  float64
 3   valeur_PM10  33824 non-null  float64
 4   valeur_PM25  39200 non-null  float64
dtypes: float64(5)
memory usage: 1.9 MB


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
id,,,,,
2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4
2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0
2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9
2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1
2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6


In [29]:
imputer_interpol = imputers.ImputerInterpolation(method="linear") #there are other options
train_imputed = imputer_interpol.fit_transform(train)



In [30]:
print("Missing values before:", train.isna().sum().sum())
print("Missing values after :", train_imputed.isna().sum().sum())

Missing values before: 25477
Missing values after : 0


In [31]:
import pandas as pd
import numpy as np

def add_time_features(df):
    """
    Adds time-based and cyclical (sin/cos) features to a DataFrame 
    when the datetime is the index.

    The index must be a DatetimeIndex.

    Features added:
    - month (1–12)
    - day (1–31)
    - week (ISO week number)
    - hour (0–23)
    - dayofweek (0=Mon, 6=Sun)
    - is_rush_hour (7–9, 16–18)
    - is_weekend (Saturday/Sunday)
    - hour_sin, hour_cos (captures 24-hour cycle)
    - dow_sin, dow_cos (captures 7-day cycle)
    - month_sin, month_cos (captures 12-month cycle)
    """

    # Ensure index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index must be a DatetimeIndex.")

    df = df.copy()
    dt = df.index  # datetime index

    # Base temporal features
    df['month'] = dt.month
    df['day'] = dt.day
    df['week'] = dt.isocalendar().week.astype(int)
    df['hour'] = dt.hour
    df['dayofweek'] = dt.dayofweek

    # Flags
    df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 16, 17, 18])
    df['is_weekend'] = df['dayofweek'] >= 5

    # Cyclical encodings
    df['hour_sin']  = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos']  = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin']   = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dow_cos']   = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    return df


Features: Rush hour, weekend

In [32]:
featured_train = add_time_features(train_imputed)
final_val = add_time_features(test)

### now doing the classic models

In [38]:
df = featured_train.copy()
split_date = df.index[int(len(df) * 0.8)]
train = df.loc[df.index <= split_date].copy()
test  = df.loc[df.index > split_date].copy()

train = featured_train.copy()

In [39]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

targets = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

# Feature columns (exclude targets)
feature_cols = [c for c in train.columns if c not in targets]

xgb_models = {}
xgb_predictions = {}
xgb_rmse = {}

for target in targets:
    print(f"Training XGBoost for {target}...")
    
    X_train, y_train = train[feature_cols], train[target]
    X_test, y_test = test[feature_cols], test[target]
    
    model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    
    xgb_models[target] = model
    xgb_predictions[target] = preds
    xgb_rmse[target] = rmse

print("\n✅ XGBoost RMSE per pollutant:")
for t, r in xgb_rmse.items():
    print(f"{t}: {r:.2f}")


Training XGBoost for valeur_NO2...
Training XGBoost for valeur_CO...
Training XGBoost for valeur_O3...
Training XGBoost for valeur_PM10...
Training XGBoost for valeur_PM25...

✅ XGBoost RMSE per pollutant:
valeur_NO2: 8.38
valeur_CO: 0.07
valeur_O3: 12.89
valeur_PM10: 6.02
valeur_PM25: 4.12


In [40]:
from prophet import Prophet

prophet_models = {}
prophet_forecasts = {}
prophet_rmse = {}

# Regressor list (must exist in train)
regressors = [c for c in feature_cols if c not in ['day', 'week']]  # exclude highly redundant ones

for target in targets:
    print(f"Training Prophet for {target}...")

    # Prepare train data
    df_train = train[[target] + regressors].reset_index().rename(columns={train.index.name: 'ds', target: 'y'})
    df_test = test[[target] + regressors].reset_index().rename(columns={test.index.name: 'ds', target: 'y'})

    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True
    )

    # Add regressors
    for reg in regressors:
        m.add_regressor(reg)

    # Fit Prophet
    m.fit(df_train)

    # Predict
    forecast = m.predict(df_test)
    preds = forecast['yhat'].values
    rmse = np.sqrt(mean_squared_error(df_test['y'], preds))

    prophet_models[target] = m
    prophet_forecasts[target] = forecast
    prophet_rmse[target] = rmse

print("\n✅ Prophet RMSE per pollutant:")
for t, r in prophet_rmse.items():
    print(f"{t}: {r:.2f}")


Training Prophet for valeur_NO2...


12:03:31 - cmdstanpy - INFO - Chain [1] start processing
12:03:58 - cmdstanpy - INFO - Chain [1] done processing


Training Prophet for valeur_CO...


12:04:01 - cmdstanpy - INFO - Chain [1] start processing
12:04:57 - cmdstanpy - INFO - Chain [1] done processing


Training Prophet for valeur_O3...


...

Training Prophet for valeur_PM10...


12:05:39 - cmdstanpy - INFO - Chain [1] start processing
12:06:38 - cmdstanpy - INFO - Chain [1] done processing


Training Prophet for valeur_PM25...


12:06:41 - cmdstanpy - INFO - Chain [1] start processing
12:07:49 - cmdstanpy - INFO - Chain [1] done processing



✅ Prophet RMSE per pollutant:
valeur_NO2: 11.75
valeur_CO: 0.09
valeur_O3: 18.36
valeur_PM10: 8.32
valeur_PM25: 5.62


In [41]:
import pandas as pd

comparison = pd.DataFrame({
    'XGBoost_RMSE': xgb_rmse,
    'Prophet_RMSE': prophet_rmse
})
print("\n📊 Model comparison:")
print(comparison)



📊 Model comparison:
             XGBoost_RMSE  Prophet_RMSE
valeur_NO2       8.378637     11.752818
valeur_CO        0.069833      0.093998
valeur_O3       12.892176     18.363763
valeur_PM10      6.015505      8.322373
valeur_PM25      4.115604      5.624207


### getting outputs

In [43]:
# Ensure datetime index is named 'id' for output
new_df = final_val.copy()
new_out = new_df.reset_index().rename(columns={new_df.index.name: 'id'})

# Features to feed to XGBoost / Prophet
feature_cols = [c for c in new_df.columns if c not in targets]


In [44]:
xgb_preds_df = pd.DataFrame({'id': new_out['id']})

for target in targets:
    xgb_model = xgb_models[target]
    xgb_preds_df[target] = xgb_model.predict(new_df[feature_cols])


In [45]:
prophet_preds_df = pd.DataFrame({'id': new_out['id']})

for target in targets:
    m = prophet_models[target]
    
    # Prepare Prophet future dataframe
    future = new_df[feature_cols].reset_index().rename(columns={new_df.index.name: 'ds'})
    
    # Prophet requires 'ds' + regressors
    for col in feature_cols:
        future[col] = new_df[col].values
    
    forecast = m.predict(future)
    prophet_preds_df[target] = forecast['yhat'].values


In [46]:
print(xgb_preds_df.head())
print(prophet_preds_df.head())


                   id  valeur_NO2  valeur_CO  valeur_O3  valeur_PM10  \
0 2024-09-03 23:00:00   28.932600   0.207108  45.468876    12.769423   
1 2024-09-04 00:00:00   32.974266   0.331565  43.776070    21.521938   
2 2024-09-04 01:00:00   31.430304   0.308422  43.264912    21.202883   
3 2024-09-04 02:00:00   29.199621   0.248249  40.615227    20.619932   
4 2024-09-04 03:00:00   30.030672   0.240734  36.850243    20.538481   

   valeur_PM25  
0     6.529727  
1     9.331450  
2     9.083836  
3     8.693399  
4     8.637000  
                   id  valeur_NO2  valeur_CO  valeur_O3  valeur_PM10  \
0 2024-09-03 23:00:00   19.729532   0.175744  53.066867    19.539062   
1 2024-09-04 00:00:00   20.786000   0.177465  49.965127    20.557204   
2 2024-09-04 01:00:00   18.710140   0.173000  50.377466    20.219284   
3 2024-09-04 02:00:00   17.464817   0.168248  50.036498    19.830325   
4 2024-09-04 03:00:00   18.154605   0.165076  47.997115    19.490111   

   valeur_PM25  
0    10.237090 

In [48]:
import pandas as pd

def format_timestamp_hour(df, timestamp_col='id'):
    """
    Reformat a datetime column to 'YYYY-MM-DD HH' format.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing a datetime column.
    timestamp_col : str, default 'id'
        Name of the datetime column to reformat.

    Returns
    -------
    pd.DataFrame
        DataFrame with the reformatted timestamp column.
    """
    df = df.copy()
    
    # Ensure column is datetime
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    
    # Format to 'YYYY-MM-DD HH' as string
    df[timestamp_col] = df[timestamp_col].dt.strftime('%Y-%m-%d %H')
    
    return df


In [49]:
format_timestamp_hour(xgb_preds_df).to_csv("xgb_new_predictions.csv", index=False)
format_timestamp_hour(prophet_preds_df).to_csv("prophet_new_predictions.csv", index=False)
